In [1]:
# these are the required modules ...
# !pip install -U pandas sqlalchemy psycopg2-binary sqlite-utils datasette datasette-pretty-json

In [2]:
# TODO:
# * ingest bib_record titles from order data
# * set an index on the "vendor.vendor_record_code 
# * build `on_order_view` to include the vendor name (from the vendor table's vendor_record_code)

In [3]:
import pandas as pd
import altair as alt
import json
import os
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, Text, JSON, Boolean, BigInteger, DateTime, Date, Numeric

# read from the config
try:
    with open("config.json", "r") as f:
        config = json.load(f)

    pg_username = config["pg_username"]
    pg_password = config["pg_password"]

except:
    exit()

engine = create_engine(
    'sqlite:///current_orders.db', 
    echo=False
)

sierra_engine = create_engine(
    'postgresql://{}:{}@sierra-db.plch.net:1032/iii'.format(
        pg_username, pg_password
    )
)

# remove the previous database
try:
    os.remove('current_orders.db')
except:
    pass

os.close(os.open('current_orders.db', os.O_CREAT))

In [4]:
# chpl_fiscal_office_vendors 
# read this data from included .csv

df = pd.read_csv('chpl_fiscal_office_vendors.csv', index_col=False)
            
df.to_sql(
    name='chpl_fiscal_office_vendors', 
    index=False, 
    if_exists='replace', 
    con=engine, 
    chunksize=10000,
    dtype={
        'FiscalVendorId':Integer(),
        'FiscalVendorCode': Text(),
        'VendorName': Text(),
        'SierraVendorCode': Text()
    }
)

6866

In [5]:
# chpl_fiscal_office_funds
# read this data from included .csv

df = pd.read_csv('chpl_fiscal_office_funds.csv', index_col=False)
            
df.to_sql(
    name='chpl_fiscal_office_funds', 
    index=False, 
    if_exists='replace', 
    con=engine, 
    chunksize=10000,
    dtype={
        'FiscalFundId': Integer(),
        'FundName': Text(),
        'SierraFundCode': Text(),
        'SierraFundCodeNumeric': Integer(),
        'SierraFundCodeSpaces': Text(),
        'GLKey': Integer(),
        'GLObj': Integer()
    }
)

91

In [6]:
# varfield_metadata

sql = """\
-- invoice, order, vendor farfiled metadata
select 
	rtn."name" as record_type_name, 
	rt.code as record_type_code,
	--rt.tag as record_type_tag,
	vt.code as varfield_code,
	vt.is_enabled as varfield_is_enabled,
	vt.display_order as varfield_display_order,
	vtn.short_name as varfield_type_short_name,
	vtn."name" as varfield_type_name 
from  
	sierra_view.varfield_type vt 
	join sierra_view.record_type rt on rt.code = vt.record_type_code
	join sierra_view.record_type_name rtn on rtn.record_type_id = rt.id 
	join sierra_view.varfield_type_name vtn on vtn.varfield_type_id = vt.id 
where rt.code in (
	'n', --INVOICE
	'o', --ORDER
	'v' --VENDOR
)
order by 
	rtn."name",
	vt.display_order 
	--record_type_code,
	--display_order 
"""

df = pd.read_sql(sql=sql, con=sierra_engine)

df.to_sql(
    name='varfield_metadata', 
    index=False, 
    if_exists='replace', 
    con=engine, 
    chunksize=10000,
    dtype={
        'record_type_name': Text(), 
        'record_type_code': Text(), 
        'varfield_code' : Text(), 
        'varfield_is_enabled': Boolean(), 
        'varfield_display_order': Integer(), 
        'varfield_type_short_name': Text() , 
        'varfield_type_name' : Text()
    }
)

58

In [7]:
# varfield
sql = """\
select 
    rm.id as record_id,
    rm.record_num,
    rm.record_type_code,
    v.id as varfield_id,
    v.varfield_type_code,
    v.occ_num,
    v.field_content 
from 
    sierra_view.record_metadata as rm 
    join sierra_view.varfield as v on v.record_id = rm.id 
where 
    rm.record_type_code in (
        'n', --INVOICE
        'o', --ORDER
        'v' --VENDOR
    )
"""

df = pd.read_sql(sql=sql, con=sierra_engine)

df.to_sql(
    name='varfield', 
    index=False, 
    if_exists='replace', 
    con=engine, 
    chunksize=10000,
    dtype={
        'record_id': BigInteger(),
        'record_num': Integer(),
        'record_type_code': Text(),
        'varfield_id': BigInteger(),
        'varfield_type_code': Text(),
        'occ_num': Integer(),
        'field_content': Text() 
    }
)

2348920

In [8]:
# %%time
# Fund data

sql = """\
SELECT 
fm.id AS fund_master_id,
au.code_num AS accounting_unit_code_num,
fm.code_num AS fund_code_num,
fm.code AS fund_code,
fmu.fund_type, -- fbal The linked data is for a current fund.
               -- oldfbal The linked data is for a fund from last year.
               -- fprevbal The linked data is for a Fund Activity Report.
fmu.fund_code,
fmu.external_fund_code_num,
efpn."name" AS external_fund_name, 
fmu.appropriation,
fmu.expenditure,
fmu.encumbrance,
fmu.num_orders,
fmu.num_payments,
fmu.warning_percent,
fmu.discount_percent,
fmu."name",
fmu.note1,
fmu.note2
FROM
sierra_view.fund_master AS fm 
JOIN sierra_view.accounting_unit AS au ON au.id = fm.accounting_unit_id
LEFT OUTER JOIN sierra_view.fund_myuser AS fmu ON fmu.fund_master_id = fm.id
LEFT OUTER JOIN sierra_view.external_fund_property AS efp ON efp.code_num = fmu.external_fund_code_num
LEFT OUTER JOIN sierra_view.external_fund_property_name AS efpn ON efpn.external_fund_property_id = efp.id 
-- sierra_view.fund_myuser AS f
ORDER BY
fm.id
"""

df = pd.read_sql(sql=sql, con=sierra_engine)

df.to_sql(
    name='fund', 
    index=False, 
    if_exists='replace', 
    con=engine, 
    chunksize=10000,
    dtype={
        'fund_master_id': BigInteger(),
        'accounting_unit_code_num': Integer(),
        'fund_code_num': Integer(),
        'fund_code': Text(),
        'fund_type': Text(),
        'fund_code': Text(),
        'external_fund_code_num': Integer(),
        'external_fund_name': Text(),
        'appropriation': Integer(),
        'expenditure': Integer(),
        'encumbrance': Integer(),
        'num_orders': Integer(),
        'num_payments': Integer(),
        'warning_percent': Integer(),
        'discount_percent': Numeric(),
        'name': Text(),
        'note1': Text(),
        'note2': Text()
    }
)

216

In [9]:
# %%time

# TODO: consider adding each line for the invoice here... 

# NOTE: we need to do a left outer join on the order record,
# since sometimes there is a credit that's not associated with
# an order record
sql = """\
WITH invoice_data AS (
    SELECT
    ir.record_id AS invoice_record_id,
    count(il.id) as count_invoice_record_line,
    json_agg(
        json_build_object( 
            --id
            --invoice_record_id
            'line_cnt', il.line_cnt,
            'order_record_id', il.order_record_metadata_id,
            'order_record_num', rm.record_num,
            'title', il.title,
            'paid_amt_cents', (il.paid_amt * 100.0) :: INTEGER,
            'lien_amt_cents', (il.lien_amt * 100.0) :: INTEGER,
            'lien_flag', il.lien_flag,
            'list_price_cents', (il.list_price * 100.0) :: INTEGER,
            'fund_code', il.fund_code,
            'subfund_num', il.subfund_num,
            'copies_paid_cnt', il.copies_paid_cnt,
            'external_fund_code_num', il.external_fund_code_num,
            'external_fund_code_num', il.status_code,
            'note', il.note,
            'is_single_copy_partial_pmt', il.is_single_copy_partial_pmt,
            'multiflag_code', il.multiflag_code,
            'line_level_tax_cents', (il.line_level_tax * 100.0) :: INTEGER,	
            'vendor_code', il.vendor_code,
            'accounting_transaction_voucher_num', il.accounting_transaction_voucher_num,
            'accounting_transaction_voucher_seq_num', il.accounting_transaction_voucher_seq_num,
            'invoice_record_vendor_summary_id', il.invoice_record_vendor_summary_id,
            'is_use_tax', il.is_use_tax,
            -- 
            -- "fairly" spread out the difference between the grand total and the subtotal among the lines based on item paid cost
            'approx_adjust_amt', 
            (
                round(
                    round(
                        (il.paid_amt / nullif(ir.subtotal_amt, 0)), -- 1. get percent of the the subtotal for the line paid amt 
                        6                                           -- 2. round to 6 decimal places for a high precision percent
                    )
                    * (coalesce(ir.grand_total_amt, 0) - coalesce(ir.subtotal_amt, 0) ), -- 3. multiply the percent above by the difference between the grand and subtotal
                    2                                                                    -- 4. round it to 2 decimal places for a dollar amount + cents approximation of the adj
                ) * 100.0 )::INTEGER                                -- 5. convert the value to cents 
            )
        ORDER BY
        il.invoice_record_id ASC,
        il.line_cnt ASC,
        il.order_record_metadata_id ASC
    ) AS json_invoice_record_line
    FROM 
    sierra_view.invoice_record AS ir 
    LEFT OUTER JOIN sierra_view.invoice_record_line AS il ON il.invoice_record_id = ir.record_id 
    LEFT OUTER JOIN sierra_view.record_metadata AS rm ON rm.id = il.order_record_metadata_id 
    --JOIN sierra_view.invoice_record_vendor_summary AS irvs ON irvs.invoice_record_id = il.invoice_record_id
    GROUP BY 1
)
SELECT
d.invoice_record_id,
rm2.record_num AS invoice_record_num,
ir2.invoice_number_text,
d.count_invoice_record_line,
d.json_invoice_record_line,
date(rm2.creation_date_gmt) AS creation_date,
date(rm2.record_last_updated_gmt) AS last_updated,
ir2.accounting_unit_code_num,
date(ir2.invoice_date_gmt) AS invoice_date,
date(ir2.paid_date_gmt) AS paid_date,
ir2.status_code AS invoice_record_status_code,
date(ir2.posted_data_gmt) AS posted_date,
ir2.is_paid_date_received_date,
-- ncode1,
-- ncode2,
-- ncode3,
ir2.iii_user_name,
ir2.foreign_currency_code,
ir2.foreign_currency_format,
ir2.foreign_currency_exchange_rate,
ir2.tax_fund_code,
ir2.tax_type_code,
(ir2.discount_amt * 100.0) :: INTEGER AS discount_amt_cents,
(ir2.grand_total_amt * 100.0) :: INTEGER AS grand_total_amt_cents,
(ir2.subtotal_amt * 100.0) :: INTEGER AS subtotal_amt_cents,
(ir2.shipping_amt * 100.0) :: INTEGER AS shipping_amt_cents,
(ir2.total_tax_amt * 100.0) :: INTEGER AS total_tax_amt_cents,
ir2.use_tax_fund_code,
ir2.use_tax_percentage_rate,
ir2.use_tax_type_code,
ir2.use_tax_ship_service_code,
ir2.is_suppressed
FROM
invoice_data AS d
JOIN sierra_view.invoice_record AS ir2 ON ir2.record_id = d.invoice_record_id
JOIN sierra_view.record_metadata rm2 ON rm2.id = d.invoice_record_id
-- 
-- Note, I'm not sure why this was needed, or is still needed, but I think it's wrong to exclude them
--
-- where ir2.grand_total_amt != ir2.subtotal_amt
--
-- ORDER BY 1 DESC
"""

df = pd.read_sql(sql=sql, con=sierra_engine)

In [10]:
# df

In [11]:
df.to_sql(
    name='invoice', 
    index=False, 
    if_exists='replace', 
    con=engine, 
    chunksize=10000,
    dtype={
        'invoice_record_id': BigInteger(),
        'invoice_record_num': Integer(),
        'invoice_number_text': Text(), 
        'count_invoice_record_line': Integer(),
        'json_invoice_record_line': JSON(),
        'creation_date': Date(),
        'last_updated': Date(),
        'accounting_unit_code_num': Integer(),
        'invoice_date': Date(),
        'paid_date': Date(),
        'invoice_record_status_code': Text(),
        'posted_date': Date(),
        'is_paid_date_received_date': Boolean(),
        'iii_user_name': Text(),
        'foreign_currency_code': Text(),
        'foreign_currency_format': Text(), 
        'foreign_currency_exchange_rate': Numeric(),
        'tax_fund_code': Text(),
        'tax_type_code': Text(),
        'discount_amt_cents': Integer(),
        'grand_total_amt_cents': Integer(),
        'subtotal_amt_cents': Integer(),
        'shipping_amt_cents': Integer(),
        'total_tax_amt_cents': Integer(),
        'use_tax_fund_code': Text(),
        'use_tax_percentage_rate': Numeric(),
        'use_tax_type_code': Text(),
        'use_tax_ship_service_code': Text(),
        'is_suppressed': Boolean()
    }
)

160164

In [12]:
sql = """\
with outer_invoice_line_data as (
	with invoice_line_data as (
		select 
		ir.record_id, 
		rm.record_num as invoice_record_num,
		il.line_cnt + 1 as line_cnt,
		(
			select 
			count(*)
			from
			sierra_view.invoice_record_line il2
			where il2.invoice_record_id = ir.record_id 
			group by il2.invoice_record_id 
		) as total_lines,
		il.paid_amt * 100.0 :: INTEGER as paid_amt_cents,
		ir.grand_total_amt * 100.0 :: INTEGER as grand_total_amt_cents ,
		ir.subtotal_amt * 100.0 :: INTEGER as subtotal_amt_cents ,
		(
		    round(
		        round(
		            (il.paid_amt / nullif(ir.subtotal_amt, 0)), -- 1. get percent of the the subtotal for the line paid amt 
		            6                                           -- 2. round to 6 decimal places for a high precision percent
		        )
		        * (coalesce(ir.grand_total_amt, 0) - coalesce(ir.subtotal_amt, 0) ), -- 3. multiply the percent above by the difference between the grand and subtotal
		        2                                                                    -- 4. round it to 2 decimal places for a dollar amount + cents approximation of the adj
		    ) * 100.0 
		)::INTEGER as approx_adj_amt                            -- 5. convert the value to cents
		from
		sierra_view.invoice_record ir 
		join sierra_view.record_metadata as rm on rm.id = ir.record_id  
		join sierra_view.invoice_record_line il on il.invoice_record_id = ir.id 
		where
		ir.posted_data_gmt is not null
		group by
		ir.record_id,
		rm.record_num,
		il.paid_amt,
		ir.subtotal_amt ,
		ir.grand_total_amt, 
		il.line_cnt 
		order by
		ir.record_id DESC,
		il.line_cnt ASC
	)
	select 
	record_id as invoice_record_id,
	invoice_record_num,
	line_cnt,
	total_lines,
	paid_amt_cents,
	grand_total_amt_cents,
	subtotal_amt_cents,
	approx_adj_amt,
	--*,
	SUM(paid_amt_cents + approx_adj_amt) over (partition by record_id order by line_cnt) as running_adjusted_total_paid_cents
	from
	invoice_line_data
)
select
invoice_record_id,
invoice_record_num,
line_cnt,
total_lines,
paid_amt_cents,
grand_total_amt_cents,
subtotal_amt_cents,
approx_adj_amt,
running_adjusted_total_paid_cents,
case
	when (line_cnt != total_lines) then NULL
	when (line_cnt = total_lines) and (running_adjusted_total_paid_cents != grand_total_amt_cents) then true
	when (line_cnt = total_lines) and (running_adjusted_total_paid_cents = grand_total_amt_cents) then false
end as is_off,
case
	when (line_cnt != total_lines) then null
	when (line_cnt = total_lines) then (grand_total_amt_cents - running_adjusted_total_paid_cents)
end as off_amt_cents
from 
outer_invoice_line_data
"""

df = pd.read_sql(sql=sql, con=sierra_engine)

df.to_sql(
    name='invoice_line_data', 
    index=False, 
    if_exists='replace', 
    con=engine, 
    chunksize=10000,
    dtype={
        'invoice_record_id' : BigInteger(),
        'line_cnt': Integer(),
        'total_lines': Integer(),
        'paid_amt_cents': Integer(),
        'grand_total_amt_cents': Integer(),
        'subtotal_amt_cents': Integer(),
        'approx_adj_amt': Integer(),
        'running_adjusted_total_paid_cents': Integer(),
        'is_off': Boolean(),
        'off_amt_cents': Integer()
    }
)

383737

In [13]:
# invoice record line

In [14]:
# %%time
# create a linking table between the invoice and order record data ...
sql = """
-- invoice_record_order_record_link
SELECT
DISTINCT
irl.invoice_record_id,
irm.record_num AS invoice_record_num,
irl.order_record_metadata_id AS order_record_id,
orm.record_num AS order_record_num
FROM 
sierra_view.invoice_record_line AS irl
JOIN sierra_view.record_metadata AS irm ON irm.id = irl.invoice_record_id 
JOIN sierra_view.record_metadata AS orm ON orm.id = irl.order_record_metadata_id
ORDER BY
irl.invoice_record_id ASC,
irl.order_record_metadata_id ASC
;
"""

df = pd.read_sql(sql=sql, con=sierra_engine)

df.to_sql(
    name='invoice_record_order_record_link', 
    index=False, 
    if_exists='replace', 
    con=engine, 
    chunksize=10000,
    dtype={
        'invoice_record_id': BigInteger(),
        'invoice_record_num': Integer(),
        'order_record_id': BigInteger(),
        'order_record_num': Integer()
    }
)

353836

In [15]:
# %%time
# create a linking table between the invoice and order record data ...
sql = """
-- invoice_record_vendor_record_link
SELECT 
s.invoice_record_id,
rm.record_num AS invoice_record_num, 
vr.record_id AS vendor_record_id,
rm2.record_num AS vendor_record_num,
s.vendor_code,
s.voucher_num,
s.voucher_total,
s.display_order
FROM 
sierra_view.invoice_record_vendor_summary AS s
LEFT OUTER JOIN sierra_view.record_metadata AS rm ON rm.id = s.invoice_record_id  
LEFT OUTER JOIN sierra_view.vendor_record AS vr ON vr.code = s.vendor_code   
LEFT OUTER JOIN sierra_view.record_metadata rm2 ON rm2.id = vr.record_id 
;
"""

df = pd.read_sql(sql=sql, con=sierra_engine)

df.to_sql(
    name='invoice_record_vendor_record_link', 
    index=False, 
    if_exists='replace', 
    con=engine, 
    chunksize=10000,
    dtype={
        'invoice_record_id': BigInteger(),
        'invoice_record_num': Integer(),
        'vendor_record_id': BigInteger(),
        'vendor_record_num': Integer(),
        'vendor_code': Text(),
        'voucher_num': Integer(),
        'voucher_total': Integer(),
        'display_order': Integer() 
    }
)

160165

In [16]:
# %%time
sql = """\
-- build the aggreate cmf data associated with the order record
WITH order_record_cmf_data AS (
    SELECT
    o_r.record_id AS order_record_id,
    count(cmf.id) AS count_cmf,
    count(DISTINCT cmf.fund_code) AS count_distinct_cmf_fund_codes,
    sum(cmf.copies) AS sum_cmf_copies,
    json_agg(
        json_build_object(
            'display_order', cmf.display_order,
            'cmf_id', cmf.id,
            'fund_code_num', fm.code_num,
            'fund_code', fm.code,
            'acct_unit_code_num', au.code_num,
            'copies', cmf.copies,
            'location_code', cmf.location_code
        )
        ORDER BY
        cmf.display_order ASC
    ) AS cmf_data
    FROM 
    sierra_view.order_record AS o_r
    LEFT OUTER JOIN sierra_view.order_record_cmf AS cmf ON cmf.order_record_id = o_r.record_id
    -- it's unfortunate, but it seems like the code number can have TEXT values like 'none' for example..
    -- so, it's necessary to filter those out with a regex
    LEFT OUTER JOIN sierra_view.fund_master AS fm ON fm.code_num = NULLIF(regexp_replace(cmf.fund_code, '[^0-9]*', '', 'g'),'')::int
    LEFT OUTER JOIN sierra_view.accounting_unit AS au ON au.id = fm.accounting_unit_id 
    WHERE 
    cmf.location_code != 'multi'
    GROUP BY 1
)
SELECT
-- build order record data
rm.record_num AS order_record_num,
d.*,
brorl.orders_display_order,
brorl.bib_record_id  AS bib_record_id,
rm2.record_num as bib_record_num,
rm.creation_date_gmt,
order_record.accounting_unit_code_num,
(order_record.estimated_price * 100.0) :: INTEGER AS estimated_price_cents,
order_record.form_code AS physical_form_code,
fpn."name" AS physical_form_name,
order_record.order_date_gmt,
order_record.catalog_date_gmt,
order_record.order_type_code,
otpm."name" AS order_type_name,
order_record.received_date_gmt,
order_record.receiving_location_code,
order_record.order_status_code,
ospn."name" AS order_status_name,
order_record.vendor_record_code,
vr.record_id AS vendor_record_id,
order_record.volume_count
FROM 
order_record_cmf_data AS d
LEFT OUTER JOIN sierra_view.order_record AS order_record ON order_record.record_id = d.order_record_id
LEFT OUTER JOIN sierra_view.record_metadata AS rm ON rm.id = order_record.record_id
LEFT OUTER JOIN sierra_view.bib_record_order_record_link AS brorl ON brorl.order_record_id = order_record.record_id
LEFT OUTER JOIN sierra_view.record_metadata AS rm2 on rm2.id = brorl.bib_record_id
LEFT OUTER JOIN sierra_view.form_property AS fp ON fp.code = order_record.form_code
LEFT OUTER JOIN sierra_view.form_property_name AS fpn ON fpn.form_property_id = fp.id 
LEFT OUTER JOIN sierra_view.order_type_property_myuser AS otpm ON otpm.code = order_record.order_type_code
LEFT OUTER JOIN sierra_view.order_status_property AS osp ON osp.code = order_record.order_status_code 
LEFT OUTER JOIN sierra_view.order_status_property_name AS ospn ON ospn.order_status_property_id = osp.id
LEFT OUTER JOIN sierra_view.vendor_record AS vr ON vr.code = order_record.vendor_record_code
"""

df = pd.read_sql(sql=sql, con=sierra_engine)

In [17]:
df.to_sql(
    name='orders', 
    index=False, 
    if_exists='replace', 
    con=engine, 
    chunksize=10000,
    dtype={
        'order_record_num': Integer(), 
        'order_record_id': BigInteger(), 
        'count_cmf': Integer(),
        'count_distinct_cmf_fund_codes': Integer(), 
        'sum_cmf_copies': Integer(), 
        'cmf_data': JSON(),
        'orders_display_order': Integer(), 
        'bib_record_id': BigInteger(), 
        'creation_date_gmt': DateTime(),
        'accounting_unit_code_num': Integer(),
        'estimated_price_cents': Integer(),
        'physical_form_code': Text(),
        'physical_form_name': Text(),
        'order_date_gmt': DateTime(),
        'catalog_date_gmt': DateTime(),
        'order_type_code': Text(),
        'order_type_name': Text(),
        'received_date_gmt': DateTime(),
        'receiving_location_code': Text(),
        'order_status_code': Text(),
        'order_status_name': Text(), 
        'vendor_record_code': Text(), 
        'vendor_record_id': BigInteger(),
        'volume_count': Integer(),        
    }
)

316473

In [18]:
sql = """\
WITH vendor_data AS (
    SELECT
    vr.record_id AS vendor_record_id,
    count(vra.id) AS count_address,
    json_agg(
        json_build_object(
          'address_type_code', vrat.code, 
            'display_order', vra.display_order,
            'addr1', vra.addr1,
            'addr2', vra.addr2,
            'addr3', vra.addr3,
            'village', vra.village,
            'city', vra.city,
            'region', vra.region,
            'postal_code', vra.postal_code,
            'country', vra.country,
            'vendor_record_address_id', vra.id 
        )
        ORDER BY
        vrat.code,
        vra.display_order ASC,
        vra.id 
    ) AS vendor_record_address 
    FROM 
    sierra_view.vendor_record AS vr
    JOIN sierra_view.vendor_record_address AS vra ON vra.vendor_record_id = vr.record_id 
    JOIN sierra_view.vendor_record_address_type AS vrat ON vrat.id = vra.vendor_record_address_type_id 
    GROUP BY 1
)
SELECT
d.vendor_record_id,
rm.record_num AS vendor_record_num,
date(rm.creation_date_gmt) AS record_create_date,
date(rm.record_last_updated_gmt) AS record_last_updated,
rm.num_revisions, 
(
    -- the first addr1 value from the first vendor address
    SELECT
    vra2.addr1 
    FROM 
    sierra_view.vendor_record_address AS vra2
    JOIN sierra_view.vendor_record_address_type vrat2 ON vrat2.id = vra2.vendor_record_address_type_id
    WHERE 
    vra2.vendor_record_id = d.vendor_record_id
    ORDER BY
    vrat2.code ASC,
    vra2.display_order ASC
    LIMIT 1
) AS vendor_name,
vr2.code AS vendor_record_code,
vr2.accounting_unit_code_num,
d.count_address,
d.vendor_record_address
FROM
vendor_data AS d
JOIN sierra_view.vendor_record vr2 ON vr2.record_id = d.vendor_record_id
JOIN sierra_view.record_metadata AS rm ON rm.id = vr2.record_id 
ORDER BY 
vr2.record_id
"""

df = pd.read_sql(sql=sql, con=sierra_engine)

In [19]:
df.to_sql(
    name='vendors', 
    index=False, 
    if_exists='replace', 
    con=engine, 
    chunksize=10000,
    dtype={
        'vendor_record_id': BigInteger(),
        'vendor_record_num': Integer(),
        'record_create_date': Date(),
        'record_last_updated': Date(),
        'num_revisions': Integer(),
        'vendor_name': Text(),
        'vendor_record_code': Text(),
        'accounting_unit_code_num': Integer(),
        'count_address': Integer(),
        'vendor_record_address': JSON() 
    }
)

5371

In [20]:
sql = """\
SELECT 
irvs.id AS invoice_record_vendor_summary_id, 
ir.invoice_number_text,
irvs.invoice_record_id,
rm.record_num AS invoice_record_num,
irvs.vendor_code,
irvs.voucher_num, 
irvs.voucher_total AS voucher_total_cents, 
irvs.display_order,
date(ir.invoice_date_gmt) AS invoice_date 
FROM 
sierra_view.invoice_record_vendor_summary AS irvs 
JOIN sierra_view.record_metadata AS rm ON rm.id = irvs.invoice_record_id
JOIN sierra_view.invoice_record AS ir ON ir.id = irvs.invoice_record_id 
ORDER BY 1 DESC
"""

df = pd.read_sql(sql=sql, con=sierra_engine)

In [21]:
df.to_sql(
    name='invoice_record_vendor_summary', 
    index=False, 
    if_exists='replace', 
    con=engine, 
    chunksize=10000,
    dtype={
        'invoice_record_vendor_summary_id': BigInteger(), 
        'invoice_number_text': Text(), 
        'invoice_record_id': BigInteger(), 
        'invoice_record_num': Integer(), 
        'vendor_code': Text(), 
        'voucher_num': Integer(), 
        'voucher_total_cents': Integer(), 
        'display_order': Integer(), 
        'invoice_date': Date() 
    }
)

160165

In [22]:
# sql = """\
# -- search for a barcode stored in the 'b' tagged varfield for the item...
# SELECT
# items.item_record_id,
# items.item_record_num,
# json_extract(value, '$.field_content') as barcode
# FROM
# items, 
# json_each(items.json_item_varfields)
# WHERE 
# json_extract(value, '$.varfield_type_code') = 'b'
# and json_extract(value, '$.field_content') like '{}'
# """

# pd.read_sql(sql=sql.format('a000073209167'), con=engine).head()

In [23]:
# !tar -cvvf - ./current_orders.db | xz -9 -T0 > current_orders.db.tar.xz

In [24]:
from sqlite_utils import Database
db = Database("current_orders.db")

db["chpl_fiscal_office_vendors"].create_index(['SierraVendorCode'])

db["chpl_fiscal_office_funds"].create_index(['SierraFundCodeSpaces'])
db["chpl_fiscal_office_funds"].create_index(['SierraFundCode'])

db["fund"].create_index(['fund_code_num', 'fund_type'])

db["orders"].create_index(['order_status_code', 'order_status_name'])
db["orders"].create_index(['physical_form_name'])
db["orders"].create_index(['bib_record_num'])
db["orders"].create_index(['order_date_gmt'])
db["orders"].create_index(['vendor_record_id'])

db["vendors"].create_index(['vendor_record_id', 'vendor_record_id'])

db["varfield"].create_index(['record_id', 'varfield_type_code'])
db["varfield"].create_index(['record_type_code', 'varfield_type_code'])

db["invoice"].create_index(['invoice_record_num'])
db["invoice"].create_index(['invoice_record_id'])
db["invoice"].create_index(['paid_date'])
db["invoice"].create_index(['posted_date'])
db["invoice"].create_index(['invoice_number_text'])

db["invoice_line_data"].create_index(['invoice_record_id', 'line_cnt'])
db["invoice_line_data"].create_index(['invoice_record_num', 'line_cnt'])

db["invoice_record_vendor_record_link"].create_index(['invoice_record_id', 'invoice_record_num'])
db["invoice_record_vendor_record_link"].create_index(['vendor_record_id', 'vendor_record_num'])


# on_order_view
sql = """\
with order_data as (
  with order_data_sub as (
    select
      DISTINCT order_record_num,
      json_extract(value, '$.fund_code') as fund_code
    from
      orders,
      json_each(orders.cmf_data)
    where
      orders.order_status_code = 'o'
      and fund_code is not null
  )
  select
    order_record_num,
    group_concat(fund_code, ', ') as fund_codes
  from
    order_data_sub
  group by
    1
)
select
  o.order_record_num,
  o.fund_codes,
  orders.count_cmf,
  orders.sum_cmf_copies,
  orders.bib_record_num,
  date(orders.creation_date_gmt) as create_date,
  orders.estimated_price_cents / 100.0 as est_price,
  (
    orders.estimated_price_cents * orders.sum_cmf_copies
  ) / 100.0 as est_total_price,
  orders.physical_form_name,
  date(orders.order_date_gmt) as order_date,
  date(orders.catalog_date_gmt) as cat_date,
  orders.order_type_name as order_type,
  date(orders.received_date_gmt) as received_date,
  orders.order_status_name,
  orders.vendor_record_code,
  orders.volume_count
from
  order_data as o
  join orders on orders.order_record_num = o.order_record_num
"""
db.create_view(
    "on_order_view", 
    sql
)


# vendors_summary_view
sql = """\
with vendors_data as (
  select
    vendors.vendor_record_num,
    vendors.vendor_record_id,
    max(orders.order_date_gmt) as max_order_date_gmt,
    count(orders.order_record_num) as count_total_order_records
  from
    vendors
    left outer join orders on orders.vendor_record_id = vendors.vendor_record_id
  group by
    1,
    2
)
select
  d.vendor_record_num,
  v.vendor_record_code,
  v.vendor_name,
  d.count_total_order_records,
  date(d.max_order_date_gmt) as last_order_date,
  v.record_create_date as vendor_record_create_date,
  v.record_last_updated as vendor_record_last_updated -- ( select orders.order_record_num from orders where orders.order_date_gmt = d.max_order_date_gmt limit 1) as last_order_record_num -- vendor_record_address
from
  vendors_data as d
  join vendors as v on v.vendor_record_id = d.vendor_record_id
order by
  last_order_date DESC
"""
db.create_view(
    "vendors_summary_view", 
    sql
)


# vendors_summary_last_invoice_view
sql = """\
with vendors_data as (
  select
    vendors.vendor_record_num,
    vendors.vendor_record_id,
    max(orders.order_date_gmt) as max_order_date_gmt,
    count(orders.order_record_num) as count_total_order_records
  from
    vendors
    left outer join orders on orders.vendor_record_id = vendors.vendor_record_id
  group by
    1,
    2
)
select
  d.vendor_record_num,
  v.vendor_record_code,
  v.vendor_name,
  d.count_total_order_records,
  date(d.max_order_date_gmt) as last_order_date,
  (
    select
      invoice.paid_date
    from
      invoice_record_vendor_record_link as l
      join invoice on invoice.invoice_record_id = l.invoice_record_id
    where
      l.vendor_record_id = d.vendor_record_id
    order by
      invoice.paid_date DESC
    limit
      1
  ) as last_invoice_paid_date,
  -- max(invoice.paid_date) as max_invoice_paid_date,
  -- date(max_invoice_paid_date) as last_invoice_paid_date,
  v.record_create_date as vendor_record_create_date,
  v.record_last_updated as vendor_record_last_updated -- ( select orders.order_record_num from orders where orders.order_date_gmt = d.max_order_date_gmt limit 1) as last_order_record_num -- vendor_record_address
from
  vendors_data as d
  join vendors as v on v.vendor_record_id = d.vendor_record_id
order by
  last_order_date DESC
"""

db.create_view(
    "vendors_summary_last_invoice_view", 
    sql
)


# 
db.create_view(
    "fiscal_office_vendors_view",
    sql="""\
select
  f.FiscalVendorId,
  f.FiscalVendorCode,
  f.VendorName,
  f.SierraVendorCode,
  v.vendor_record_num,
  v.vendor_record_id,
  v.record_create_date,
  v.record_last_updated,
  (
    select
      date(max(orders.order_date_gmt))
    from
      orders
    where
      orders.vendor_record_id = v.vendor_record_id
  ) as max_order_date
from
  chpl_fiscal_office_vendors as f
  left outer join vendors as v on v.vendor_record_code = f.SierraVendorCode
order by
  f.FiscalVendorId
  """
)


sql = """\
with union_data as (
  with invoice_data as (
    select
      invoice_record_id,
      invoice_record_num,
      invoice_number_text,
      posted_date,
      invoice_date,
      json_extract(value, '$.line_cnt') + 1 as line_cnt,
      json_extract(value, '$.paid_amt_cents') / 100.0 as paid_amount,
      -- * 100.0 as paid_amount,
      json_extract(value, '$.vendor_code') as vendor_code
    from
      invoice,
      json_each(invoice.json_invoice_record_line) --    where
      --      "posted_date" >= :start_date
      --       and "posted_date" <= :end_date
  )
  select
    id1.invoice_record_id,
    id1.invoice_record_num,
    id1.invoice_number_text,
    id1.posted_date,
    id1.invoice_date,
    id1.line_cnt,
    id1.paid_amount,
    id1.vendor_code,
    null as discount_amt,
    null as grand_total_amt,
    null as subtotal_amt
  from
    invoice_data as id1
  union all
  select
    distinct id2.invoice_record_id,
    id2.invoice_record_num,
    id2.invoice_number_text,
    id2.posted_date,
    id2.invoice_date,
    0 as line_cnt,
    null as paid_amount,
    id2.vendor_code,
    i.discount_amt_cents / 100.0,
    i.grand_total_amt_cents / 100.0,
    i.subtotal_amt_cents / 100.0
  from
    invoice_data as id2
    join invoice as i on i.invoice_record_id = id2.invoice_record_id
)
select
  invoice_record_id,
  invoice_record_num,
  invoice_number_text,
  posted_date,
  invoice_date,
  case
    line_cnt
    when 0 then null
    else line_cnt
  end as line_cnt,
  paid_amount,
  vendor_code,
  discount_amt,
  grand_total_amt,
  subtotal_amt
from
  union_data
order by
  invoice_date,
  invoice_number_text,
  line_cnt
  """


db.create_view(
    "munis_invoice_output", 
    sql
)



# peid_view
sql = """\
-- PeId
-- vendor code coming from a Sierra varfield associated with the vendor record--
-- either 'd'('Alternate Code') OR 'z' -- ('Special') -- prefer 'd' over 'z'
-- if there's no value in Sierra, fall-back to the fiscal office version with the
-- coalesce function
select
  distinct invoice_record_id,
  invoice_record_num,
  invoice_number_text,
  coalesce (
    (
      select
        case
          -- this is baffling to me. when I have the varfield type code = 'd' the query takes a long time,
          -- if i put this case statement here, it finishes quickly
          varfield_type_code
          when 'z' then NULL
          else varfield.field_content
        end
      from
        vendors
        join varfield on varfield.record_id = vendors.vendor_record_id
      where
        trim(json_extract(value, '$.vendor_code')) = vendors.vendor_record_code
        and varfield_type_code in ('d', 'z')
      order by
        varfield_type_code ASC,
        -- prefer 'd' OVER 'z'
        occ_num ASC
      limit
        1
    ), (
      select
        FiscalVendorCode
      from
        chpl_fiscal_office_vendors
      where
        SierraVendorCode = trim(json_extract(value, '$.vendor_code'))
      limit
        1
    )
  ) as peid
from
  invoice,
  json_each(invoice.json_invoice_record_line)
  left outer join vendors on vendors.vendor_record_code = json_extract(value, '$.vendor_code')
"""

db.create_view(
    "peid_view", 
    sql
)



# ref_view
sql = """\
--ref
select
  invoice.invoice_record_id,
  invoice.invoice_number_text as invoice_number_text_orig,
  case
    -- this is a test to see if we can remove the leading zeros if the value is all numeric
    -- (e.g. 0001234 -> 1234)
    -- (e.g. 00890CO22422086 -> 00890CO22422086)
    when length(cast(invoice_number_text as integer)) = length(ltrim(invoice_number_text, 0)) then ltrim(invoice_number_text, 0)
    else invoice_number_text
  end as ref
from
  invoice
"""

db.create_view(
    "ref_view", 
    sql
)



# this was the previous munis output ... leaving this in here for future consideration 
# sql = """
# select
#   -- HDR ROW TYPE / DTL ROW TYPE
#   -- This is either the grand total of the invoice record, or the individual invoice record line data ... the first run through will be all "data" lines
#   'D' as "HDR ROW TYPE / DTL ROW TYPE",
#   -- "HDR VENDOR NUMBER / DTL VENDOR NUMBER"
#   -- AKA "PeId"
#   -- vendor code coming from a Sierra varfield associated with the vendor record--
#   -- either 'd'('Alternate Code') OR 'z' -- ('Special') -- prefer 'd' over 'z'
#   -- if there's no value in Sierra, fall-back to the fiscal office version with the
#   -- coalesce function
#   coalesce (
#     (
#       select
#         case
#           -- this is baffling to me. when I have the varfield
#           -- type code = 'd' the query takes a long time,
#           -- if i put this case statement here, it finishes quickly
#           varfield_type_code
#           when 'z' then NULL
#           else varfield.field_content
#         end
#       from
#         vendors
#         join varfield on varfield.record_id = vendors.vendor_record_id
#       where
#         trim(json_extract(value, '$.vendor_code')) = vendors.vendor_record_code
#         and varfield_type_code in ('d', 'z')
#       order by
#         varfield_type_code ASC,
#         -- prefer 'd' OVER 'z'
#         occ_num ASC
#       limit
#         1
#     ), (
#       select
#         FiscalVendorCode
#       from
#         chpl_fiscal_office_vendors
#       where
#         SierraVendorCode = trim(json_extract(value, '$.vendor_code'))
#       limit
#         1
#     )
#   ) as "HDR VENDOR NUMBER / DTL VENDOR NUMBER",
#   -- invoice_record_id,
#   -- invoice_record_num,
#   --
#   -- "HDR INVOICE NUMBER / DTL INVOICE NUMBER"
#   case
#     -- this is a test to see if we can remove the leading
#     -- zeros if the value is all numeric
#     -- (e.g. 0001234 -> 1234)
#     -- (e.g. 00890CO22422086 -> 00890CO22422086)
#     when length(cast(invoice_number_text as integer)) = length(ltrim(invoice_number_text, 0)) then ltrim(invoice_number_text, 0)
#     else invoice_number_text
#   end as "HDR INVOICE NUMBER / DTL INVOICE NUMBER",
#   -- "HDR INVOICE TOTAL / DTL SEQUENCE"
#   -- this first go around will be the line_cnt value from the invoice record,
#   -- the second go around will be the grand_total_amt_cents
#   json_extract(value, '$.line_cnt') + 1 as "HDR INVOICE TOTAL / DTL SEQUENCE",
#   -- 'GEN' as "Div",
#   -- "HDR INVOICE NET AMOUNT / DTL AMOUNT"
#   -- I'm not sure yet if the NET AMOUNT is related to something stored in Sierra, or if this is some sort of running total
#   json_extract(value, '$.paid_amt_cents') / 100.0 as "HDR INVOICE NET AMOUNT / DTL AMOUNT",
#   -- 'AP' as "CheckId",
#   -- CheckId I believe only is applicable when it's the HDR ...
#   NULL as "CheckId",
#   -- Not sure if this needs to be NULL the first time through, or if this should be only filled in if it's related to the HDR
#   -- invoice_date as RefDt,
#   NULL as RefDt,
#   -- I need to look into where to grab this information --maybe off the order record?
#   NULL as "DTL DESCRIPTION",
#   fo.GLKey as "GlKey",
#   fo.GLObj as "GlObj",
#   -- AltAddrCd maybe only needed when it's the HDR row
#   NULL as "AltAddrCd",
#   NULL as "_extra_data -->",
#   cast(
#     json_extract(value, '$.fund_code') as integer
#   ) as fund_code,
#   posted_date,
#   grand_total_amt_cents / 100.0 as grand_total_amt,
#   -- discount_amt_cents / 100.0 as discount_amt,
#   -- subtotal_amt_cents / 100.0 as subtotal_amt,
#   -- shipping_amt_cents / 100.0 as shipping_amt,
#   -- total_tax_amt_cents / 100.0 as total_tax_amt,
#   json_extract(value, '$.line_cnt') + 1 as line_cnt,
#   json_extract(value, '$.vendor_code') as vendor_code
# from
#   invoice,
#   json_each(invoice.json_invoice_record_line)
#   left outer join vendors on vendors.vendor_record_code = json_extract(value, '$.vendor_code')
#   left outer join fund on (
#     fund.fund_code_num = cast(
#       json_extract(value, '$.fund_code') as integer
#     )
#     and fund_type = 'fbal' -- this is the current fund
#   ) -- match on the fund code name
#   left outer join chpl_fiscal_office_funds as fo on lower(fo.SierraFundCode) = lower(fund.fund_code)
# where
#   -- "HDR INVOICE NUMBER / DTL INVOICE NUMBER" = :invoice_number --
#   -- "posted_date" >= :invoice_posted_start_date
#   -- and "posted_date" <= :invoice_posted_end_date -- and
#   -- and 
#   "paid_amt_cents" != 0
# order by
#   "PeId",
#   -- "RefDt",
#   "Ref",
#   line_cnt;
# """

# db.create_view(
#     "munis_output_view", 
#     sql
# )



# # dev_munis_output_view
# sql = """\
# with invoice_data as (
#   select
#     invoice_record_id,
#     invoice_record_num,
#     invoice_number_text,
#     count_invoice_record_line,
#     invoice_date,
#     paid_date,
#     invoice_record_status_code,
#     posted_date,
#     grand_total_amt_cents,
#     subtotal_amt_cents,
#     (
#       select
#         peid
#       from
#         peid_view
#       where
#         peid_view.invoice_record_id = invoice.invoice_record_id
#       limit
#         1
#     ) as peid,
#     (
#       select
#         ref
#       from
#         ref_view
#       where
#         ref_view.invoice_record_id = invoice.invoice_record_id
#       limit
#         1
#     ) as ref
#   from
#     invoice
#   -- where
#   --  invoice.invoice_number_text = '2031452593'
# )
# select
#   'D' as "HDR ROW TYPE / DTL ROW TYPE",
#   -- PeId
#   invoice_data.peid as "HDR VENDOR NUMBER / DTL VENDOR NUMBER",
#   -- Ref
#   invoice_data.ref as "HDR INVOICE NUMBER / DTL INVOICE NUMBER",
#   json_extract(value, '$.line_cnt') + 1 as "HDR INVOICE TOTAL / DTL SEQUENCE",
#   -- json_extract(value, '$.paid_amt_cents') / 100.0 as "HDR INVOICE NET AMOUNT / DTL AMOUNT",
#   case
#     when invoice_line_data.line_cnt = invoice_line_data.total_lines then (
#       invoice_line_data.approx_adj_amt + invoice_line_data.paid_amt_cents + invoice_line_data.off_amt_cents
#     ) / 100.0 -- else, this isn't the last line, and we can just add the approximation to the paid amt
#     else (
#       invoice_line_data.approx_adj_amt + invoice_line_data.paid_amt_cents
#     ) / 100.0
#   end as "HDR INVOICE NET AMOUNT / DTL AMOUNT",
#   '' as "HDR PAYMENT METHOD",
#   invoice_data.invoice_date as "HDR INVOICE DATE",
#   --
#   -- DLT DESCRIPTION is defined as follows 
#   -- 5410- PRINT MATERIALS, 5420- PERIODICALS, 5430- A/V LIBRARY MATERIALS, 5450- ELECTRONIC MATERIALS, 5210- ADMIN SUPPLIES
#   CASE
#     WHEN fo.GLObj = 5410 THEN 'PRINT MATERIALS'
#     WHEN fo.GLObj = 5420 THEN 'PERIODICALS'
#     WHEN fo.GLObj = 5430 THEN 'A/V LIBRARY MATERIALS'
#     WHEN fo.GLObj = 5450 THEN 'ELECTRONIC MATERIALS'
#     WHEN fo.GLObj = 5210 THEN 'ADMIN SUPPLIES'
#   end as "DTL DESCRIPTION",
#   -- GlKey
#   fo.GLKey as "DTL ORG",
#   -- GlObj
#   fo.GLObj as "DTL OBJECT",
#   '' as "_extra_data -->",
#   json_extract(value, '$.paid_amt_cents') / 100.0 as "orig_paid_amt",
#   invoice_data.*
# from
#   invoice_data
#   join invoice on invoice.invoice_record_id = invoice_data.invoice_record_id,
#   json_each(invoice.json_invoice_record_line) --
#   left outer join invoice_line_data on (
#     invoice_data.invoice_record_id = invoice_line_data.invoice_record_id
#     AND json_extract(value, '$.line_cnt') + 1 = invoice_line_data.line_cnt
#   )
#   left outer join fund on (
#     fund.fund_code_num = cast(
#       json_extract(value, '$.fund_code') as integer
#     )
#     and fund_type = 'fbal' -- this is the current fund
#   ) -- match on the fund code name
#   left outer join chpl_fiscal_office_funds as fo on lower(fo.SierraFundCode) = lower(fund.fund_code)
# union all
# select
#   'H' as "HDR ROW TYPE / DTL ROW TYPE",
#   -- PeId
#   invoice_data.peid as "HDR VENDOR NUMBER / DTL VENDOR NUMBER",
#   -- Ref
#   invoice_data.ref as "HDR INVOICE NUMBER / DTL INVOICE NUMBER",
#   grand_total_amt_cents / 100.0 as "HDR INVOICE TOTAL / DTL SEQUENCE",
#   grand_total_amt_cents / 100.0 as "HDR INVOICE NET AMOUNT / DTL AMOUNT",
#   '' as "HDR PAYMENT METHOD",
#   invoice_data.invoice_date as "HDR INVOICE DATE",
#   NULL as "DTL DESCRIPTION",
#   -- GlKey
#   NULL as "DTL ORG",
#   -- GlObj
#   NULL as "DTL OBJECT",
#   --
#   -- extra data
#   '' as "_extra_data -->",
#   null as "orig_paid_amt",
#   invoice_data.*
# from
#   invoice_data --invoice2
# order by
#   invoice_date ASC,
#   invoice_record_id ASC,
#   "HDR ROW TYPE / DTL ROW TYPE" DESC -- invoice record line from the json object
# """

# db.create_view(
#     "dev_munis_output_view", 
#     sql
# )



# for ease / speed, lets also just create this data as a table
# (REuse the query from above, but don't create it as a view)
sql = """\
CREATE table munis_output as 
with invoice_data as (
  select
    invoice_record_id,
    invoice_record_num,
    invoice_number_text,
    count_invoice_record_line,
    invoice_date,
    paid_date,
    invoice_record_status_code,
    posted_date,
    grand_total_amt_cents,
    subtotal_amt_cents,
    (
      select
        peid
      from
        peid_view
      where
        peid_view.invoice_record_id = invoice.invoice_record_id
      limit
        1
    ) as peid,
    (
      select
        ref
      from
        ref_view
      where
        ref_view.invoice_record_id = invoice.invoice_record_id
      limit
        1
    ) as ref
  from
    invoice
  -- where
  --  invoice.invoice_number_text = '2031452593'
)
select
  'D' as "HDR ROW TYPE / DTL ROW TYPE",
  -- PeId
  invoice_data.peid as "HDR VENDOR NUMBER / DTL VENDOR NUMBER",
  -- Ref
  invoice_data.ref as "HDR INVOICE NUMBER / DTL INVOICE NUMBER",
  json_extract(value, '$.line_cnt') + 1 as "HDR INVOICE TOTAL / DTL SEQUENCE",
  -- json_extract(value, '$.paid_amt_cents') / 100.0 as "HDR INVOICE NET AMOUNT / DTL AMOUNT",
  case
    when invoice_line_data.line_cnt = invoice_line_data.total_lines then (
      invoice_line_data.approx_adj_amt + invoice_line_data.paid_amt_cents + invoice_line_data.off_amt_cents
    ) / 100.0 -- else, this isn't the last line, and we can just add the approximation to the paid amt
    else (
      invoice_line_data.approx_adj_amt + invoice_line_data.paid_amt_cents
    ) / 100.0
  end as "HDR INVOICE NET AMOUNT / DTL AMOUNT",
  --
  -- Removing this line, and combining with the next column ...
  -- '' as "HDR PAYMENT METHOD",
  -- 
  -- this line is getting moved as well
  -- invoice_data.invoice_date as "HDR INVOICE DATE",
  --
  -- DLT DESCRIPTION is defined as follows 
  -- 5410- PRINT MATERIALS, 5420- PERIODICALS, 5430- A/V LIBRARY MATERIALS, 5450- ELECTRONIC MATERIALS, 5210- ADMIN SUPPLIES
  CASE
    WHEN fo.GLObj = 5410 THEN 'PRINT MATERIALS'
    WHEN fo.GLObj = 5420 THEN 'PERIODICALS'
    WHEN fo.GLObj = 5430 THEN 'A/V LIBRARY MATERIALS'
    WHEN fo.GLObj = 5450 THEN 'ELECTRONIC MATERIALS'
    WHEN fo.GLObj = 5210 THEN 'ADMIN SUPPLIES'
  end as "HDR PAYMENT METHOD / DTL DESCRIPTION",
  -- GlKey
  fo.GLKey as "HDR INVOICE DATE/DTL ORG",
  -- GlObj
  fo.GLObj as "HDR DUE DATE/DTL OBJECT",
  '' as "_extra_data -->",
  json_extract(value, '$.paid_amt_cents') / 100.0 as "orig_paid_amt",
  invoice_data.*
from
  invoice_data
  join invoice on invoice.invoice_record_id = invoice_data.invoice_record_id,
  json_each(invoice.json_invoice_record_line) --
  left outer join invoice_line_data on (
    invoice_data.invoice_record_id = invoice_line_data.invoice_record_id
    AND json_extract(value, '$.line_cnt') + 1 = invoice_line_data.line_cnt
  )
  left outer join fund on (
    fund.fund_code_num = cast(
      json_extract(value, '$.fund_code') as integer
    )
    and fund_type = 'fbal' -- this is the current fund
  ) -- match on the fund code name
  left outer join chpl_fiscal_office_funds as fo on lower(fo.SierraFundCode) = lower(fund.fund_code)


UNION ALL

-- Generate the HDR row

select
  'H' as "HDR ROW TYPE / DTL ROW TYPE",
  -- PeId
  invoice_data.peid as "HDR VENDOR NUMBER / DTL VENDOR NUMBER",
  -- Ref
  invoice_data.ref as "HDR INVOICE NUMBER / DTL INVOICE NUMBER",
  grand_total_amt_cents / 100.0 as "HDR INVOICE TOTAL / DTL SEQUENCE",
  grand_total_amt_cents / 100.0 as "HDR INVOICE NET AMOUNT / DTL AMOUNT",
  NULL as "HDR PAYMENT METHOD / DTL DESCRIPTION",
  --
  -- this line is getting removed / moved
  -- invoice_data.invoice_date as "HDR INVOICE DATE",
  --
  --
  -- NULL as "DTL DESCRIPTION",
  --
  
  -- GlKey
  strftime('%m/%d/%Y', invoice_data.invoice_date) as "HDR INVOICE DATE/DTL ORG",
  -- GlObj
  strftime('%m/%d/%Y', invoice_data.invoice_date) as "HDR DUE DATE/DTL OBJECT",
  --
  -- extra data
  '' as "_extra_data -->",
  null as "orig_paid_amt",
  invoice_data.*
from
  invoice_data --invoice2
order by
  invoice_date ASC,
  invoice_record_id ASC,
  "HDR ROW TYPE / DTL ROW TYPE" DESC -- invoice record line from the json object
"""

db.execute(sql)

# create the indexes on some key fileds ....
db["munis_output"].create_index(['invoice_date'])
db["munis_output"].create_index(['posted_date'])
db["munis_output"].create_index(['HDR INVOICE NUMBER / DTL INVOICE NUMBER'])
db["munis_output"].create_index(['HDR VENDOR NUMBER / DTL VENDOR NUMBER'])


# recreate the top part as a view so that we can more easily format it for form input entry
sql = """\
select 
*
from
munis_output
"""

db.create_view(
    "munis_output_view", 
    sql
)




# one_solution_output_view
sql = """\
select
  -- PeId
  -- vendor code coming from a Sierra varfield associated with the vendor record--
  -- either 'd'('Alternate Code') OR 'z' -- ('Special') -- prefer 'd' over 'z'
  -- if there's no value in Sierra, fall-back to the fiscal office version with the
  -- coalesce function
  coalesce (
    (
      select
        case
          -- this is baffling to me. when I have the varfield type code = 'd' the query takes a long time,
          -- if i put this case statement here, it finishes quickly
          varfield_type_code
          when 'z' then NULL
          else varfield.field_content
        end
      from
        vendors
        join varfield on varfield.record_id = vendors.vendor_record_id
      where
        trim(json_extract(value, '$.vendor_code')) = vendors.vendor_record_code
        and varfield_type_code in ('d', 'z')
      order by
        varfield_type_code ASC,
        -- prefer 'd' OVER 'z'
        occ_num ASC
      limit
        1
    ), (
      select
        FiscalVendorCode
      from
        chpl_fiscal_office_vendors
      where
        SierraVendorCode = trim(json_extract(value, '$.vendor_code'))
      limit
        1
    )
  ) as PeId,
  -- invoice_record_id,
  -- invoice_record_num,
  case
    -- this is a test to see if we can remove the leading zeros if the value is all numeric
    -- (e.g. 0001234 -> 1234)
    -- (e.g. 00890CO22422086 -> 00890CO22422086)
    when length(cast(invoice_number_text as integer)) = length(ltrim(invoice_number_text, 0)) then ltrim(invoice_number_text, 0)
    else invoice_number_text
  end as "Ref",
  'GEN' as "Div",
  -- when we hit the final line, add any leftover from the approximation to that last line
  case
    when invoice_line_data.line_cnt = invoice_line_data.total_lines then (
      invoice_line_data.approx_adj_amt + invoice_line_data.paid_amt_cents + invoice_line_data.off_amt_cents
    ) / 100.0 -- else, this isn't the last line, and we can just add the approximation to the paid amt
    else (
      invoice_line_data.approx_adj_amt + invoice_line_data.paid_amt_cents
    ) / 100.0
  end as DistAmt,
  --0.0 as DistAmt,paid_amt_cents
  -- json_extract(value, '$.paid_amt_cents') / 100.0 as "DistAmt",
  'AP' as "CheckId",
  invoice_date as "RefDt",
  fo.GLKey,
  fo.GLObj,
  'B1' as "AltAddrCd",
  NULL as "_extra_data -->",
  invoice.invoice_record_id,
  invoice.invoice_record_num,
  json_extract(value, '$.line_cnt') + 1 as line_cnt,
  count_invoice_record_line,
  json_extract(value, '$.paid_amt_cents') / 100.0 as "OrigDistAmt",
  cast(
    json_extract(value, '$.fund_code') as integer
  ) as fund_code,
  posted_date,
  invoice.grand_total_amt_cents,
  -- discount_amt_cents / 100.0 as discount_amt,
  invoice.subtotal_amt_cents,
  -- shipping_amt_cents / 100.0 as shipping_amt,
  -- total_tax_amt_cents / 100.0 as total_tax_amt,
  json_extract(value, '$.vendor_code') as vendor_code,
  invoice_line_data.approx_adj_amt,
  invoice_line_data.running_adjusted_total_paid_cents,
  invoice_line_data.off_amt_cents
from
  invoice,
  json_each(invoice.json_invoice_record_line) -- left join here to get the adjusted amounts
  left outer join invoice_line_data on (
    invoice.invoice_record_id = invoice_line_data.invoice_record_id
    AND json_extract(value, '$.line_cnt') + 1 = invoice_line_data.line_cnt
  )
  left outer join vendors on vendors.vendor_record_code = json_extract(value, '$.vendor_code')
  left outer join fund on (
    fund.fund_code_num = cast(
      json_extract(value, '$.fund_code') as integer
    )
    and fund_type = 'fbal' -- this is the current fund
  ) -- match on the fund code name
  left outer join chpl_fiscal_office_funds as fo on lower(fo.SierraFundCode) = lower(fund.fund_code)
where
  -- "posted_date" >= :invoice_posted_start_date
  -- and "posted_date" <= :invoice_posted_end_date
  -- and
  "paid_amt_cents" != 0
  and "fund_code" != '65535'
order by
  "PeId",
  -- "RefDt",
  "Ref",
  line_cnt
"""
db.create_view(
    "one_solution_output_view", 
    sql
)



sql = """\
with invoice_data as (
  select
    *
  from
    invoice
  where
    posted_date is not null --  where
    --    posted_date >= '2022-01-01'
    --    and posted_date < '2023-01-01'
)
select
  trim(json_extract(value, '$.vendor_code')) as vendor_code,
  vendors.vendor_name,
  vendors.vendor_record_num,
  v.field_content as "Alternate Code",
  count(distinct invoice_record_id) as count_invoices,
  max(invoice_data.posted_date) as last_posted_date
from
  invoice_data,
  json_each(invoice_data.json_invoice_record_line)
  left outer join vendors on vendors.vendor_record_code = vendor_code --
  left outer join varfield as v on (
    v.record_id = vendors.vendor_record_id
    and v.varfield_type_code = 'd'
  )
group by
  1,
  2,
  3,
  4
order by
  5 DESC
"""

db.create_view(
    "invoice_vendor_alt_code_view", 
    sql
)

<Database <sqlite3.Connection object at 0x7ff455e3fe40>>

In [25]:
# Generate the fund use chart for the front page ...
sql = """\
SELECT
ifnull(round(
    ((fund.expenditure + fund.encumbrance)*1.0 / (NULLIF(appropriation, 0) * 1.0))*100.0,
    2
), 0) as total_percent_fund_used,
ifnull(round(
    ((fund.expenditure)*1.0 / (NULLIF(appropriation, 0) * 1.0))*100.0,
    2
), 0) as " expenditure % used",
ifnull(round(
    ((fund.encumbrance)*1.0 / (NULLIF(appropriation, 0) * 1.0))*100.0,
    2
), 0) as "encumbrance % used",
coalesce(external_fund_name, '') 
    || ' / ' 
    || coalesce(fund_code, '') as "fund name / fund code",
fund_code,
appropriation,
encumbrance,
expenditure
-- *
FROM
fund
WHERE
fund.fund_type = 'fbal'
AND appropriation > 0
-- ORDER BY percent_fund_used DESC
"""

df = pd.read_sql(sql=sql, con=engine)
# df.head()

# melt "expenditure_percent_fund_used" "encumbrance_percent_fund_used" into "fund use type" ...
out_df = df.melt(
    id_vars=["total_percent_fund_used", "fund name / fund code", "appropriation", "encumbrance", "expenditure"],  
    value_vars=[" expenditure % used", "encumbrance % used"],
    var_name="fund use type", 
    value_name="percent"
)

# out_df.head()

# place the chart into a variable
chart = alt.Chart(out_df).mark_bar().encode(    
    x=alt.X('percent',),
    y=alt.Y('fund name / fund code', sort='-x'),
    color='fund use type',
    tooltip=['total_percent_fund_used', 'fund name / fund code', 'appropriation', 'encumbrance', 'expenditure', 'fund use type', 'percent']
    # tooltip=['percent_fund_used', 'fund_name', 'fund_code', 'appropriation', 'encumbrance', 'expenditure']
    # tooltip=['transaction_date', 'transaction_location_code', 'op_name', 'count'],
    #color='op_name'
).properties(
    title = 'Fund Use ( as of {} )'.format(pd.Timestamp('now').strftime('%Y-%m-%d %I:00 %p'))
    #title='Total Checkouts Per Location - {}'.format(search_date)
).configure_title(
    # fontSize=20,
    # anchor='start',
)

chart.save('./my-app/static/current-fund-use.html')

# display the chart
# HTML('output.html')

/home/plchuser/order-exploder/venv/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


In [26]:
# remove old db, and replace it with the new 
# TODO: restart datasette
!rm my-app/current_orders.db
!ln current_orders.db my-app/current_orders.db

In [27]:
# build the metadata.json for datasette ...
# canned queries look like this in the metadata.json file
"""
{
    "databases": {
       "sf-trees": {
           "queries": {
               "just_species": {
                   "sql": "select qSpecies from Street_Tree_List"
               }
           }
       }
    }
}
"""

metadata = {
    'title': 'CHPL Order Exploder 💣',
    'extra_css_urls': ['/static/my.css'],
    'databases': {
        'current_orders': {
            'queries': {}
        }
    }
}

# query invoice_with_order
metadata['databases']['current_orders']['queries']['invoice_with_order'] = {}
metadata['databases']['current_orders']['queries']['invoice_with_order']['sql'] = """\
SELECT
  i.*
from
  invoice_record_order_record_link as l
  join invoice as i on i.invoice_record_num = l.invoice_record_num
where
  l.order_record_num = :order_record_num
order by
  i.last_updated DESC
"""
metadata['databases']['current_orders']['queries']['invoice_with_order']['hide_sql'] = True

# query on_order_fund_code
metadata['databases']['current_orders']['queries']['on_order_fund_code'] = {}
metadata['databases']['current_orders']['queries']['on_order_fund_code']['sql'] = """\
SELECT
  orders.*
FROM
  orders,
  json_each(orders.cmf_data)
WHERE
  json_extract(value, '$.fund_code') = :fund_code
  and orders.order_status_code = 'o'
"""
metadata['databases']['current_orders']['queries']['on_order_fund_code']['hide_sql'] = True


# # query on_order_fund_code
# metadata['databases']['current_orders']['queries']['on_order_fund_code'] = {}
# metadata['databases']['current_orders']['queries']['on_order_fund_code']['sql'] = """\
# metadata['databases']['current_orders']['queries']['on_order_fund_code']['hide_sql'] = True
# """


print(json.dumps(metadata, indent=4))
with open('my-app/metadata.json', 'w') as f:
    f.write(json.dumps(metadata))

{
    "title": "CHPL Order Exploder \ud83d\udca3",
    "extra_css_urls": [
        "/static/my.css"
    ],
    "databases": {
        "current_orders": {
            "queries": {
                "invoice_with_order": {
                    "sql": "SELECT\n  i.*\nfrom\n  invoice_record_order_record_link as l\n  join invoice as i on i.invoice_record_num = l.invoice_record_num\nwhere\n  l.order_record_num = :order_record_num\norder by\n  i.last_updated DESC\n",
                    "hide_sql": true
                },
                "on_order_fund_code": {
                    "sql": "SELECT\n  orders.*\nFROM\n  orders,\n  json_each(orders.cmf_data)\nWHERE\n  json_extract(value, '$.fund_code') = :fund_code\n  and orders.order_status_code = 'o'\n",
                    "hide_sql": true
                }
            }
        }
    }
}
